In [16]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_union

In [17]:
#p_test.SentimentText=p_test.SentimentText.astype(str)
train = pd.read_csv('data/trainfile_r3.csv').astype(str)
#train.SentimentText=train.SentimentText.astype(str)
test = pd.read_csv('data/testfile_r3.csv').astype(str)
#test.SentimentText=test.SentimentText.astype(str)
Sentiment = pd.get_dummies(train['Sentiment'])
Sentiment = Sentiment.rename( columns={0.0: "Positive", 1.0: "Negative",2.0:"Neutral"})
train = train.join(Sentiment)
train.drop(['Sentiment'], axis=1, inplace=True)

In [18]:
train['Positive'] = train['0.0']
train['Negative'] = train['1.0']
train['Neutral'] = train['2.0']
train.drop(['0.0'], axis=1, inplace=True)
train.drop(['1.0'], axis=1, inplace=True)
train.drop(['2.0'], axis=1, inplace=True)

In [19]:
class_names = ['Positive','Negative','Neutral']


train_text = train['Text']
test_text = test['Text']
all_text = pd.concat([train_text, test_text])

word_vectorizer = TfidfVectorizer(
    #lowercase = False,
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 1),
    max_features=30000)
char_vectorizer = TfidfVectorizer(
   # lowercase = False,
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    max_features=30000)
vectorizer = make_union(word_vectorizer, char_vectorizer, n_jobs=2)

vectorizer.fit(all_text)
trn_term_doc = vectorizer.transform(train_text)
test_term_doc  = vectorizer.transform(test_text)

In [20]:
label_cols = ['Positive','Negative','Neutral']
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)
x = trn_term_doc
test_x = test_term_doc
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r
preds_train = np.zeros((len(train), len(label_cols)))

for i, j in enumerate(label_cols):
    print('fit for train data', j)
    m,r = get_mdl(train[j])
    preds_train[:,i] = m.predict_proba(x.multiply(r))[:,1]

preds_test = np.zeros((len(test), len(label_cols)))
for i, j in enumerate(label_cols):
    print('fit for test data', j)
    m,r = get_mdl(train[j])
    preds_test[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

fit for train data Positive
fit for train data Negative
fit for train data Neutral
fit for test data Positive
fit for test data Negative
fit for test data Neutral


In [21]:
tt = []
for i in range(len(train)):
    tt.append(i)
submid = pd.DataFrame({'Id': tt})
prediction_train = pd.concat([submid, pd.DataFrame(preds_train, columns = label_cols)], axis=1)

In [22]:
submid = pd.DataFrame.from_dict({'Id': test['Id'], })                                   
prediction_test = pd.concat([submid, pd.DataFrame(preds_test, columns = label_cols)], axis=1)

In [23]:
read = prediction_train
result = []
for i in range(len(read['Positive'])):
    if(read['Positive'][i] > read['Negative'][i] and read['Positive'][i] > read['Neutral'][i]):
        result.append(0.0)
    elif(read['Negative'][i] > read['Positive'][i] and read['Negative'][i] > read['Neutral'][i]):
        result.append(1.0)
    else:
        result.append(2.0)
ac = pd.read_csv('data/trainfile_r3.csv')
true = 0.0
for i in range(15000):
    if(result[i]==ac["Sentiment"][i]):
        true += 1
print("checking accuracy on train dataset.")
print(true/15000.0)

checking accuracy on train dataset.
0.9280666666666667


In [24]:
read = prediction_test
result = []
for i in range(len(read['Positive'])):
    if(read['Positive'][i] > read['Negative'][i] and read['Positive'][i] > read['Neutral'][i]):
        result.append(0.0)
    elif(read['Negative'][i] > read['Positive'][i] and read['Negative'][i] > read['Neutral'][i]):
        result.append(1.0)
    else:
        result.append(2.0)
read['Sentiment'] = np.array(result).T
read.drop(['Positive'], axis=1, inplace=True)
read.drop(['Negative'], axis=1, inplace=True)
read.drop(['Neutral'], axis=1, inplace=True)
read.to_csv('data/submission.csv', index=False)